#**Setup**

---



In [1]:
!ls

sample_data


In [ ]:
!git clone https://<username>:<password>@github.com/ai4bharat/indic-bert
%cd indic-bert
!pip3 install -r requirements.txt
%cd ..
!mkdir iglue outputs models

#**Configuration Variables**


---



In [ ]:
MODEL_NAME = 'models/indic-bert'
TASK_NAME = 'agc'
TRAIN_LANG = 'kn'
TEST_LANG = 'kn'


#**Download Models and Datasets**
---


In [33]:

# !pip3 install google google-cloud-storage

import os

from google.cloud import storage

# modify to specify other models, datasets. Paths must end with '/'
model_path, model_name = 'multilingual/all/albert-base-orig-full-final/', 'indic-bert'
dataset_path, dataset_name = 'wnli-translated/', 'indicnlp-articles'

bucket_name = 'nlp-corpora--ai4bharat'
bucket_store_key = 'indic-bert/keys/ai4b-gcp-key.json'
client = storage.Client.from_service_account_json(bucket_store_key)
bucket = client.get_bucket(bucket_name)

def download(remote_path, download_path):
  blobs = client.list_blobs(bucket_name, prefix=remote_path, delimiter=None)
  filenames = []
  for blob in blobs:
    filename = blob.name[blob.name.rfind(remote_path)+len(remote_path):]
    filename = os.path.join(download_path, filename)
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    filenames.append(filename)
    blob.download_to_filename(filename)

download('indicnlp-datasets/evaluation/' + dataset_path, 'iglue/' + dataset_name)
# download('indicnlp-bert/' + model_path, 'models/' + model_name)


#**Fine-tune the Model**
---



In [ ]:

import os

from fine_tune.cli import main as finetune_main

argvec = [
          '--train_lang', 'gu',
          '--test_lang', 'gu',
          '--task', 'agc',
          '--model_name_or_path', 'bert-base-multilingual-cased',
          '--config_name', '',
          '--tokenizer_name', '',
          '--data_dir', '../iglue/indicnlp-articles',
          '--output_dir', '../outputs',
          '--max_seq_length', '128',
          '--learning_rate', '2e-5',
          '--num_train_epochs', '3',
          '--train_batch_size', '32',
          '--seed', '2',
          '--n_gpu', '1',
          '--do_train',
          '--do_predict'
]

finetune_main(argvec)

#**Check the Results**
---